In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
from html.parser import HTMLParser

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # library to handle JSON files
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    beautifulsoup4-4.9.0       |   py36h9f0ad1d_0         160 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.6 MB

The following NEW packages will be INSTALLED:

    python_abi:      3.6-1_cp36m       conda-forge

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1                

# 1.Scraping Data from Wikipedia

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "html.parser")

# 2.Prepare Dataframe

In [8]:
gdp_table = soup.find("table", attrs={"class": "wikitable"})
gdp_table_data = gdp_table.tbody.find_all("tr")  # contains 2 rows

# Get all the headings of Lists
headings = []
for td in gdp_table_data[0].find_all("th"):
    #print(td.text)
    # remove any newlines and extra spaces from left and right
    headings.append(td.text.replace('\n', ' ').strip())

print(headings)
data = {}
table_data = []
for tr in gdp_table.tbody.find_all("tr"): # find all tr's from table's tbody
        t_row = {}

        # find all td's(3) in tr and zip it with t_header
        for td, th in zip(tr.find_all("td"), headings): 
            t_row[th] = td.text.replace('\n', '').strip()
        table_data.append(t_row)

# Put the data for the table with his heading.
testdata = pd.DataFrame(table_data)

['Postal code', 'Borough', 'Neighborhood']


## 2.a.remove not assigned Borough

In [9]:
onlyassigned = testdata[testdata['Borough'] != "Not assigned"]
onlyassigned = onlyassigned.dropna()

## 2.b.replace / with , in neighborhood

In [10]:
onlyassigned["Neighborhood"].replace("/",",", regex=True, inplace=True)

## 2.c. display the shape of the dataframe

In [11]:
onlyassigned.shape

(103, 3)